In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [4]:
df = pd.read_csv(r"C:\Users\manda\OneDrive\Desktop\Internship works\Credit card Fraud detection project\creditcard.csv")

In [5]:
df.shape

(284807, 31)

In [6]:
df.tail()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [7]:
scaler = StandardScaler()
df[['Time', 'Amount']] = scaler.fit_transform(df[['Time', 'Amount']])

In [8]:
X = df.drop('Class', axis=1)
y = df['Class']


In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(contamination=0.0017, random_state=42)
iso_forest.fit(X_train)

# Predict anomalies on test set
y_pred_if = iso_forest.predict(X_test)
# Convert prediction: -1 → fraud (1), 1 → legit (0)
y_pred_if = [1 if x == -1 else 0 for x in y_pred_if]

In [11]:
from sklearn.svm import LinearSVC

In [12]:
svm_clf = LinearSVC(random_state=42, max_iter=10000)
svm_clf.fit(X_train, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,42


In [15]:
from sklearn.metrics import classification_report

print("Isolation Forest classification report:")
print(classification_report(y_test, y_pred_if))

Isolation Forest classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.31      0.34      0.32        98

    accuracy                           1.00     56962
   macro avg       0.66      0.67      0.66     56962
weighted avg       1.00      1.00      1.00     56962



In [16]:
from sklearn.metrics import roc_auc_score

roc_auc_svm = roc_auc_score(y_test, svm_clf.decision_function(X_test))
print("SVM ROC-AUC:", roc_auc_svm)


SVM ROC-AUC: 0.9431332043228096


In [17]:
roc_auc_if = roc_auc_score(y_test, -iso_forest.decision_function(X_test))
print("Isolation Forest ROC-AUC:", roc_auc_if)


Isolation Forest ROC-AUC: 0.9543364475784686


In [18]:
import pickle

# Save Isolation Forest model
with open("isolation_forest.pkl", "wb") as f:
    pickle.dump(iso_forest, f)

# Save Linear SVC model
with open("linear_svc.pkl", "wb") as f:
    pickle.dump(svm_clf, f)


In [19]:
pickle.dump(X_test, open('X_test.pkl', 'wb'))
pickle.dump(y_test, open('y_test.pkl', 'wb'))
pickle.dump(svm_clf.decision_function(X_test), open('svm_scores.pkl', 'wb'))
pickle.dump(-iso_forest.decision_function(X_test), open('iso_scores.pkl', 'wb'))
